In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dense
from tensorflow.keras.optimizers import Adam

In [4]:
# Load the data from CSV file
data = pd.read_csv(r"C:\Users\janak\ConvLSTM\Daily_HAI_Mean.csv")

In [5]:
data.head()

,ds,Lat,Lon,y,unique_id
0,2020-01-01,51.728440,7.333657,0.117984,1
1,2020-01-02,51.745312,7.324808,0.128913,1
2,2020-01-03,51.736941,7.322903,0.123177,1
3,2020-01-04,51.736578,7.307931,0.125445,1
4,2020-01-05,51.744239,7.330141,0.113853,1


In [6]:
# Preprocess the data
# Assuming you have latitude, longitude, and activity_index columns
# Convert date to datetime format
data['date'] = pd.to_datetime(data['ds'])
# Normalize latitude and longitude
data['latitude'] = (data['Lat'] - data['Lat'].min()) / (data['Lat'].max() - data['Lat'].min())
data['longitude'] = (data['Lon'] - data['Lon'].min()) / (data['Lon'].max() - data['Lon'].min())
data['activity_index'] = (data['y'] - data['y'].min()) / (data['y'].max() - data['y'].min())



In [7]:
data.head()

,ds,Lat,Lon,y,unique_id,date,latitude,longitude
0,2020-01-01,51.728440,7.333657,0.117984,1,2020-01-01,0.399589,0.344182
1,2020-01-02,51.745312,7.324808,0.128913,1,2020-01-02,0.612946,0.244590
2,2020-01-03,51.736941,7.322903,0.123177,1,2020-01-03,0.507084,0.223151
3,2020-01-04,51.736578,7.307931,0.125445,1,2020-01-04,0.502494,0.054646
4,2020-01-05,51.744239,7.330141,0.113853,1,2020-01-05,0.599377,0.304618


In [9]:
data['y'].min(), data['y'].max()

(0.0972100438685852, 0.1314913564989662)

In [ ]:

# Define parameters
seq_length = 7  # Length of sequence for each sample
n_features = 3  # Number of features (latitude, longitude, activity_index)
n_filters = 32  # Number of filters in ConvLSTM layers
kernel_size = (3, 3)  # Kernel size for ConvLSTM layers

# Prepare the data for ConvLSTM
X = []
y = []
for i in range(len(data) - seq_length):
    X.append(data[['latitude', 'longitude', 'activity_index']].iloc[i:i+seq_length].values)
    y.append(data['activity_index'].iloc[i+seq_length])
X = np.array(X)
y = np.array(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for ConvLSTM
X_train = X_train.reshape(-1, seq_length, 1, n_features, 1)
X_test = X_test.reshape(-1, seq_length, 1, n_features, 1)

# Build the ConvLSTM model
model = Sequential([
    ConvLSTM2D(filters=n_filters, kernel_size=kernel_size, activation='relu', input_shape=(seq_length, 1, n_features, 1)),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(), loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
